## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [2]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
    if agg['user__username__count'] >= 9:
         users.append(agg['user__username'])   
users

['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi']

In [3]:

base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

100

¿Cuántos etiquetamos todes?

In [4]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
amoro,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Busquemos los que están etiquetados por todos

In [5]:
df = df.loc[["jmperez", "jzajac", "mkondra"]]

df = df[df.columns[df.notna().all()]]

df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
from nltk import agreement

coders = df.loc[["jzajac", "jmperez", "mkondra"]].values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.11645847769732925

Ergh...bueno

## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [7]:
users = [
    "jmperez", 
    "mkondra", 
    "ndebandi",
    "jzajac",
    "amoro",
]

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,311633,311634,311635,311636,311637,311638,311639,311640,311641,311642,311643,311644,311645,311646,311647,311648,311649,311650,311651,311652,...,311844,311845,311846,311847,311848,311849,311850,311851,311852,311853,311854,311855,311856,311857,311858,311859,311860,311861,311862,311863
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mkondra,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ndebandi,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jzajac,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amoro,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from nltk import agreement

def kappa_of(subset):
    coders = df_comments.loc[subset]
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    [u for u in users if u != "jzajac"],
    ["jmperez", "mkondra", "ndebandi", "jzajac"],
    ["jmperez", "mkondra", "ndebandi"],
    ["jmperez", "mkondra", "jzajac"],
    ["jmperez", "mkondra", "amoro"],
    ["mkondra", "ndebandi", "jzajac"],
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para jmperez mkondra ndebandi jzajac amoro    = 0.435 (#instancias = 251)
Fleiss K para jmperez mkondra ndebandi amoro           = 0.537 (#instancias = 251)
Fleiss K para jmperez mkondra ndebandi jzajac          = 0.414 (#instancias = 273)
Fleiss K para jmperez mkondra ndebandi                 = 0.526 (#instancias = 273)
Fleiss K para jmperez mkondra jzajac                   = 0.466 (#instancias = 688)
Fleiss K para jmperez mkondra amoro                    = 0.547 (#instancias = 251)
Fleiss K para mkondra ndebandi jzajac                  = 0.417 (#instancias = 521)


Pares


Fleiss K para jmperez mkondra                          = 0.616 (#instancias = 688)
Fleiss K para jmperez ndebandi                         = 0.438 (#instancias = 273)
Fleiss K para jmperez jzajac                           = 0.443 (#instancias = 688)
Fleiss K para jmperez amoro                            = 0.421 (#instancias = 251)
Fleiss K para mkondra ndebandi                         = 0.542 (#instancias =

In [12]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
jmperez     204.0
mkondra     165.0
ndebandi     96.0
jzajac      481.0
amoro        84.0
dtype: float64

## Agreements sobre cada categoría

In [13]:
users = [
    "jmperez", 
    "mkondra", 
    "ndebandi",
    "jzajac",
    "amoro",
]


idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)

df_labels

Empty DataFrame
Columns: []
Index: [(MUJER, jmperez), (MUJER, mkondra), (MUJER, ndebandi), (MUJER, jzajac), (MUJER, amoro), (LGBTI, jmperez), (LGBTI, mkondra), (LGBTI, ndebandi), (LGBTI, jzajac), (LGBTI, amoro), (RACISMO, jmperez), (RACISMO, mkondra), (RACISMO, ndebandi), (RACISMO, jzajac), (RACISMO, amoro), (POBREZA, jmperez), (POBREZA, mkondra), (POBREZA, ndebandi), (POBREZA, jzajac), (POBREZA, amoro), (POLITICA, jmperez), (POLITICA, mkondra), (POLITICA, ndebandi), (POLITICA, jzajac), (POLITICA, amoro), (DISCAPACIDAD, jmperez), (DISCAPACIDAD, mkondra), (DISCAPACIDAD, ndebandi), (DISCAPACIDAD, jzajac), (DISCAPACIDAD, amoro), (ASPECTO, jmperez), (ASPECTO, mkondra), (ASPECTO, ndebandi), (ASPECTO, jzajac), (ASPECTO, amoro), (CRIMINAL, jmperez), (CRIMINAL, mkondra), (CRIMINAL, ndebandi), (CRIMINAL, jzajac), (CRIMINAL, amoro), (OTROS, jmperez), (OTROS, mkondra), (OTROS, ndebandi), (OTROS, jzajac), (OTROS, amoro)]

In [14]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

df_labels

311633 311634 311635 311636 311637 311638 311639  \
categoria    etiquetador                                                    
MUJER        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
LGBTI        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
RACISMO      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
POBREZA      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
POLITICA     jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
DISCAPACIDAD jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
ASPECTO      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
CRIMINAL     jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
OTROS        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   

                         311640 311641 311642 311643 311644 311645 311646  \
categoria    etiquetador                                                    
MUJER        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac 

In [15]:

def kappa_of(df):
    
    coders = df[df.columns[df.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


print("##", ' - '.join(users), '\n')

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## jmperez - mkondra - ndebandi - jzajac - amoro 

| MUJER          |  0.393
No hay elementos para calcular Kappa de LGBTI - salteando
| RACISMO        |  0.597
| POBREZA        |  0.145
| POLITICA       |  0.181
No hay elementos para calcular Kappa de DISCAPACIDAD - salteando
| ASPECTO        |  0.454
| CRIMINAL       |  0.768
No hay elementos para calcular Kappa de OTROS - salteando
Instancias: 251




### jmperez - mkondra - jzajac 

| Category      | Agreement |
|---------------|-----------|
| MUJER        |  0.461    |
| LGBTI        |  0.645    |
| RACISMO      |  0.600    |
| POBREZA      |  0.387    |
| POLITICA     |  0.309    |
| DISCAPACIDAD | 0.253     |
| ASPECTO      |  0.512    |
| CRIMINAL     |  0.728    |
| OTROS        |  -0.002   |
Instancias: 688

### mkondra - jzajac 
| Category      | Agreement |
|---------------|-----------|
| MUJER       |  0.303    |
| LGBTI       |  0.503    |
| RACISMO     |  0.510    |
| POBREZA     |  0.364    |
| POLITICA    |  0.154    |
| DISCAPACIDAD|  0.098    |
| ASPECTO     |  0.460    |
| CRIMINAL    |  0.669    |
| OTROS       |  -0.004   |

Instancias: 986

### jmperez - mkondra 
| Category      | Agreement |
|---------------|-----------|
| MUJER          |  0.601   |   
| LGBTI          |  0.754   |
| RACISMO        |  0.715   |
| POBREZA        |  0.230   |
| POLITICA       |  0.289   |
| DISCAPACIDAD   |  0.758   |
| ASPECTO        |  0.699   |
| CRIMINAL       |  0.850   |
| OTROS          |  0.000   |
Instancias: 688

### jmperez - mkondra - ndebandi - jzajac 

| Category      | Agreement |
|---------------|-----------|
| MUJER         |     0.386 |
| RACISMO       |     0.541 |
| POBREZA       |     0.202 |
| POLITICA      |     0.308 |
| DISCAPACIDAD  |     0.280 |
| ASPECTO       |     0.606 |
| CRIMINAL      |     0.730 |

Instancias: 273

## MUJER


In [19]:

users = [
    #"jmperez",
    "mkondra",
    "jzajac",
]

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

def show_from_df(df, show=False):
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)

        article = comment.article

        print("="*80 + "\n")

        print(article.title, "\n")
        print(comment.text)
        
        if show:
            print(df[comment_id])

In [20]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    disagree_comments = category_df[category_df.columns[no_agree]]
    
    return agree_comments, disagree_comments


agree_comments, disagree_comments = get_agrees_and_disagrees(df_labels, "MUJER", users)

## Ejemplos de odio:

Como regla general, si un comentario ofensivo podría ser usado casi indistintamente para atacar a un hombre, entonces no debería ser considerado discriminatorio contra las mujeres


In [21]:
%%html

<blockquote class="twitter-tweet"><p lang="es" dir="ltr">Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” <a href="https://t.co/0yLfUkUacL">https://t.co/0yLfUkUacL</a></p>&mdash; infobae (@infobae) <a href="https://twitter.com/infobae/status/1296507657189314566?ref_src=twsrc%5Etfw">August 20, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>



### Ejemplos discriminatorios:

- La marcha fue p q dejen d avasallar los derechos y la democracia y p defender la República d esta panda d ladrones y enchufados ineptos y faltos d talento como esta **rubia tarada** q actúa igual hace 40 años! No a la reforma judicial y demás atropellos!

- No me sorprende toda la vida hizo el papel de la pelotuda con gomas
- Pero callate si te acostabas con Fito! 

- Miauuuuu 

### Ejemplos no discriminatorios (aunque posiblemente ofensivos):
- *No existió ninguna marcha anticuarentena....boba* **(podría decírsele a un hombre también)**
- *RECORDAS KUANDO VENDIAS JUNTO A FITO KILOS DE MERKA ????*
- *Otra forra* **(agresivo pero a priori no misógino)**
- *Y a quién le importa?* **agresivo, no misógino**
- *Vieja merquera* **(la discrimina por vieja, por adicta, pero no por mujer)**
- *No se ni quien es, pero sin conocerla puedo decir que es una estúpida. Irresponsabilidad es lo que esta haciendo este gobierno fascista con el pueblo* 

### Ejemplos a revisar

- *Y vos un foco infeccioso después de comerte al murciélago de Feto 🤮🤮🤮🤮*
- *De la mía no vas a comer nunca descerebrada, hasta las publicidades en las q apareces cambio.*
- *Cara gorda,cara flaca,igual cara de traste*
- *Y a quién choto le puede importar la opinión de ésta mina ?*


In [22]:
show_from_df(agree_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION @Sonia43917925 Cuál marcha anti cuarentena? 
La marcha fue p q dejen d avasallar los derechos y la democracia y p defender la República d esta panda d ladrones y enchufados ineptos y faltos d talento como esta rubia tarada q actúa igual hace 40 años!
No a la reforma judicial y demás atropellos!

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Chuparle la pija a Fito Páez fue una irresponsabilidad sanitaria, payasa. De qué hablás??

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Le prestaste la cachucha a Feto Paez Roth!!!🤮🤮 eso si que es una "irresponsabilidad sanitaria"

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Otra imbesil kaka! Anda hacerte ortear!

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@inf

Busquemos dónde hay discrepancias

In [23]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Que hace esta mina o sra no se
etiquetador
mkondra    False
jzajac      True
Name: 294791, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Quien es esta vieja del agua?
etiquetador
mkondra    False
jzajac      True
Name: 294794, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION A mí vos me pareces una pelotuda importante
etiquetador
mkondra    False
jzajac      True
Name: 294800, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Ahora puedo dormir tranquilo. La científica infectóloga y doctora en medicina, Cecilia Roth, dijo la verdad de la milanesa.
etiquetador
mkondra    False
jzajac      True
Name: 294805, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 



## LGBTI


In [91]:
category_df = df_labels.loc["LGBTI"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [92]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Novarecio puto gorilon.

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Viejo maricota

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Qué payasa ...

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Aféitense, chicas! 💋

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Que par de putos.

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Uno mas idiota que el otro manga de t...

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina

In [93]:
show_from_df(no_agree_comments, show=True)


Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom Naa pero vayanse a cagar hijos de remil puta, y yo aca respetando la cuarentena como un boludo, me pueden chupar bien la pija ahora
etiquetador
mkondra    False
jzajac      True
Name: 262082, dtype: object

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom La única cuestión de imagen que les jode a ustedes es la de Brasil , Chile, Bolivia, y toda América al lado nuestro. Chupenla.
etiquetador
mkondra    False
jzajac      True
Name: 262100, dtype: object

"Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero 

@LANACION Encima parece heterosexual! 
Sino puede entrar a cualquier lado en tetas que está bien..
etiquetador
mkondra    False
jzajac      True
Name: 296590, dtype: object

Luis Novaresio contó cómo será su boda con Bra

In [17]:
category_df = df_labels.loc["OTROS"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [18]:
show_from_df(agree_comments)

In [19]:
show_from_df(no_agree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Nadie la juzga por su adicción a las drogas. Que no venga a criticar las expresiones de voluntad.
etiquetador
mkondra    False
jzajac      True
Name: 294796, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Simplemente no me interesa lo que diga una adicta
etiquetador
mkondra    False
jzajac      True
Name: 294813, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Esta señora grupo de riesgo debería cuidarse de no excederse con la fafafa en vez de usar el dedito acusador.
etiquetador
mkondra    False
jzajac      True
Name: 294822, dtype: object

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION RECORDAS KUANDO VENDIAS JUNTO A FITO KILOS DE MERKA ????
etiquetador
mkondra    False
jzajac      True
Name: 294827, dtype: object

